# Úvod do prompt inžinierstva
Prompt inžinierstvo je proces navrhovania a optimalizácie promptov pre úlohy spracovania prirodzeného jazyka. Zahŕňa výber vhodných promptov, ladenie ich parametrov a hodnotenie ich výkonnosti. Prompt inžinierstvo je kľúčové pre dosiahnutie vysokej presnosti a efektivity modelov NLP. V tejto časti sa pozrieme na základy prompt inžinierstva s využitím modelov OpenAI na prieskum.


### Cvičenie 1: Tokenizácia
Preskúmajte tokenizáciu pomocou tiktoken, rýchleho open-source tokenizátora od OpenAI.
Viac príkladov nájdete v [OpenAI Cookbook](https://github.com/openai/openai-cookbook/blob/main/examples/How_to_count_tokens_with_tiktoken.ipynb?WT.mc_id=academic-105485-koreyst).


In [ ]:
# EXERCISE:
# 1. Run the exercise as is first
# 2. Change the text to any prompt input you want to use & re-run to see tokens

import tiktoken

# Define the prompt you want tokenized
text = f"""
Jupiter is the fifth planet from the Sun and the \
largest in the Solar System. It is a gas giant with \
a mass one-thousandth that of the Sun, but two-and-a-half \
times that of all the other planets in the Solar System combined. \
Jupiter is one of the brightest objects visible to the naked eye \
in the night sky, and has been known to ancient civilizations since \
before recorded history. It is named after the Roman god Jupiter.[19] \
When viewed from Earth, Jupiter can be bright enough for its reflected \
light to cast visible shadows,[20] and is on average the third-brightest \
natural object in the night sky after the Moon and Venus.
"""

# Set the model you want encoding for
encoding = tiktoken.encoding_for_model("gpt-3.5-turbo")

# Encode the text - gives you the tokens in integer form
tokens = encoding.encode(text)
print(tokens);

# Decode the integers to see what the text versions look like
[encoding.decode_single_token_bytes(token) for token in tokens]

### Cvičenie 2: Overenie nastavenia kľúča Github Models

Spustite nasledujúci kód, aby ste overili, že váš endpoint Github Models je správne nastavený. Kód jednoducho vyskúša základný prompt a overí odpoveď. Vstup `oh say can you see` by sa mal doplniť niečo ako `by the dawn's early light..`


In [2]:
import os
from azure.ai.inference import ChatCompletionsClient
from azure.ai.inference.models import SystemMessage, UserMessage
from azure.core.credentials import AzureKeyCredential

token = os.environ["GITHUB_TOKEN"]
endpoint = "https://models.inference.ai.azure.com"

model_name = "gpt-4o"

client = ChatCompletionsClient(
    endpoint=endpoint,
    credential=AzureKeyCredential(token),
)

def get_completion(prompt, client, model_name, temperature=1.0, max_tokens=1000, top_p=1.0):
    response = client.complete(
        messages=[
            {
                "role": "system",
                "content": "You are a helpful assistant.",
            },
            {
                "role": "user",
                "content": prompt,
            },
        ],
        model=model_name,
        temperature=temperature,
        max_tokens=max_tokens,
        top_p=top_p
    )
    return response.choices[0].message.content

## ---------- Call the helper method

### 1. Set primary content or prompt text
text = f"""
oh say can you see
"""

### 2. Use that in the prompt template below
prompt = f"""
```{text}```
"""

## 3. Run the prompt
response = get_completion(prompt, client, model_name)
print(response)


That line is the opening lyric of "The Star-Spangled Banner," the national anthem of the United States, written by Francis Scott Key. If you'd like more information or analysis, feel free to ask!


### Cvičenie 3: Vymyslené informácie
Preskúmajte, čo sa stane, keď požiadate LLM, aby vrátil odpovede na výzvu o téme, ktorá možno neexistuje, alebo o témach, o ktorých nemusí vedieť, pretože boli mimo jeho predtrénovaného datasetu (novšie informácie). Sledujte, ako sa odpoveď zmení, ak použijete inú výzvu alebo iný model.


In [ ]:

## Set the text for simple prompt or primary content
## Prompt shows a template format with text in it - add cues, commands etc if needed
## Run the completion 
text = f"""
generate a lesson plan on the Martian War of 2076.
"""

prompt = f"""
```{text}```
"""

response = get_completion(prompt, client, model_name)
print(response)

### Cvičenie 4: Na základe inštrukcie  
Použite premennú "text" na nastavenie hlavného obsahu  
a premennú "prompt" na zadanie inštrukcie týkajúcej sa tohto hlavného obsahu.

Tu žiadame model, aby zhrnul text pre druháka základnej školy


In [4]:
# Test Example
# https://platform.openai.com/playground/p/default-summarize

## Example text
text = f"""
Jupiter is the fifth planet from the Sun and the \
largest in the Solar System. It is a gas giant with \
a mass one-thousandth that of the Sun, but two-and-a-half \
times that of all the other planets in the Solar System combined. \
Jupiter is one of the brightest objects visible to the naked eye \
in the night sky, and has been known to ancient civilizations since \
before recorded history. It is named after the Roman god Jupiter.[19] \
When viewed from Earth, Jupiter can be bright enough for its reflected \
light to cast visible shadows,[20] and is on average the third-brightest \
natural object in the night sky after the Moon and Venus.
"""

## Set the prompt
prompt = f"""
Summarize content you are provided with for a second-grade student.
```{text}```
"""

## Run the prompt
response = get_completion(prompt, client, model_name)
print(response)

Jupiter is the fifth planet from the Sun and the biggest one in our Solar System. It's made of gas and is much bigger than all the other planets put together! You can see Jupiter in the night sky because it's very bright. People have noticed it for a really long time and named it after a Roman god.


### Cvičenie 5: Komplexný prompt
Vyskúšajte požiadavku, ktorá obsahuje správy systému, používateľa a asistenta
Systém nastavuje kontext asistenta
Správy používateľa a asistenta poskytujú kontext viacotáčkovej konverzácie

Všimnite si, ako je osobnosť asistenta v systémovom kontexte nastavená na „sarkastickú“.
Skúste použiť iný kontext osobnosti. Alebo vyskúšajte inú sériu vstupných/výstupných správ


In [5]:
response = client.complete(
    model=model_name,
    messages=[
        {"role": "system", "content": "You are a sarcastic assistant."},
        {"role": "user", "content": "Who won the world series in 2020?"},
        {"role": "assistant", "content": "Who do you think won? The Los Angeles Dodgers of course."},
        {"role": "user", "content": "Where was it played?"}
    ]
)
print(response.choices[0].message.content)

Oh, you mean the famous 2020 World Series that wasn’t in a regular location? That was the year they played in the glamorous Arlington, Texas, at Globe Life Field.


### Cvičenie: Preskúmajte svoju intuíciu
Vyššie uvedené príklady vám ukazujú vzory, ktoré môžete použiť na vytváranie nových promptov (jednoduchých, zložitých, inštrukčných atď.) – skúste vytvoriť ďalšie cvičenia, aby ste preskúmali niektoré ďalšie nápady, o ktorých sme hovorili, ako napríklad príklady, nápovedy a podobne.



---

**Vyhlásenie o vylúčení zodpovednosti**:  
Tento dokument bol preložený pomocou AI prekladateľskej služby [Co-op Translator](https://github.com/Azure/co-op-translator). Hoci sa snažíme o presnosť, upozorňujeme, že automatizované preklady môžu obsahovať chyby alebo nepresnosti. Za autoritatívny zdroj sa považuje pôvodný dokument v jeho natívnom jazyku. Pre kritické informácie odporúčame profesionálny ľudský preklad. Nezodpovedáme za žiadne nedorozumenia alebo nesprávne interpretácie vyplývajúce z použitia tohto prekladu.
